In [1]:
'''
NOTE - This notebook is implemented in Google Colab 
(Everything is sequential)
'''

'\nNOTE - This notebook is implemented in Google Colab \n(Everything is sequential)\n'

BAYESIAN OPTIMIZATION

In [2]:
import pandas as pd
import numpy as np

In [3]:
df = pd.read_csv('drive/MyDrive/Colab Files/HP_tuning/diabetes.csv')

In [4]:
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [5]:
# Filing the missing values

In [6]:
df['Glucose'] = np.where(df['Glucose']==0,df['Glucose'].mean(),df['Glucose'])
df['BloodPressure'] = np.where(df['BloodPressure']==0,df['BloodPressure'].mean(),df['BloodPressure'])
df['Insulin'] = np.where(df['Insulin']==0,df['Insulin'].mean(),df['Insulin'])
df['SkinThickness'] = np.where(df['SkinThickness']==0,df['SkinThickness'].mean(),df['SkinThickness'])

In [7]:
df.head(10)

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148.0,72.000000,35.000000,79.799479,33.6,0.627,50,1
1,1,85.0,66.000000,29.000000,79.799479,26.6,0.351,31,0
2,8,183.0,64.000000,20.536458,79.799479,23.3,0.672,32,1
3,1,89.0,66.000000,23.000000,94.000000,28.1,0.167,21,0
4,0,137.0,40.000000,35.000000,168.000000,43.1,2.288,33,1
5,5,116.0,74.000000,20.536458,79.799479,25.6,0.201,30,0
6,3,78.0,50.000000,32.000000,88.000000,31.0,0.248,26,1
7,10,115.0,69.105469,20.536458,79.799479,35.3,0.134,29,0
8,2,197.0,70.000000,45.000000,543.000000,30.5,0.158,53,1
9,8,125.0,96.000000,20.536458,79.799479,0.0,0.232,54,1


In [8]:
# Seperating Dependent and Independent Variable

In [9]:
X = df.drop(['Outcome'],axis=1)
y = df['Outcome']

In [10]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=0)

In [11]:
# Information about hyperopt

In [12]:
from hyperopt import hp,fmin,tpe,STATUS_OK,Trials

In [13]:
# Creating the space
space = {
    'criterion':hp.choice('criterion',['entropy','gini']),
    'max_depth':hp.quniform('max_depth',10,1200,10),
    'max_features':hp.choice('max_features',['auto','sqrt','log2',None]),
    'min_samples_leaf':hp.uniform('min_samples_leaf',0,0.5),
    'min_samples_split':hp.uniform('min_samples_split',0,1),
    'n_estimators':hp.choice('n_estimators',[10,50,300,750,1200,1400])
}

In [14]:
space

{'criterion': <hyperopt.pyll.base.Apply at 0x7f2d3e4016d0>,
 'max_depth': <hyperopt.pyll.base.Apply at 0x7f2d3e401b50>,
 'max_features': <hyperopt.pyll.base.Apply at 0x7f2d3e401c90>,
 'min_samples_leaf': <hyperopt.pyll.base.Apply at 0x7f2d3e401f50>,
 'min_samples_split': <hyperopt.pyll.base.Apply at 0x7f2d3e403110>,
 'n_estimators': <hyperopt.pyll.base.Apply at 0x7f2d3e403290>}

In [15]:
from sklearn.ensemble import RandomForestClassifier

In [16]:
# Defining the objective function
# Note - since we want to maximize accuracy, we are minimizing loss as loss = -accuracy

In [17]:
def objective(space):
    model = RandomForestClassifier(n_estimators=space['n_estimators'],criterion=space['criterion'],
                                    max_depth=space['max_depth'],max_features=space['max_features'],
                                    min_samples_leaf=space['min_samples_leaf'],
                                    min_samples_split=space['min_samples_split'])
    
    accuracy = cross_val_score(model,X_train,y_train,cv=5).mean()
    return {'loss': -accuracy, 'status': STATUS_OK}

In [18]:
from sklearn.model_selection import cross_val_score
trials  = Trials()
best = fmin(fn = objective,
            space = space,
            algo = tpe.suggest,
            max_evals=80,
            trials = trials,
            )
best


100%|██████████| 80/80 [07:28<00:00,  5.61s/it, best loss: -0.7719712115153938]


{'criterion': 0,
 'max_depth': 240.0,
 'max_features': 2,
 'min_samples_leaf': 0.11044560463320821,
 'min_samples_split': 0.16792381603680173,
 'n_estimators': 5}

In [19]:
# The output of the best after running on all the iterations

'''{'criterion': 0,
 'max_depth': 240.0,
 'max_features': 2,
 'min_samples_leaf': 0.11044560463320821,
 'min_samples_split': 0.16792381603680173,
 'n_estimators': 5}
'''

"{'criterion': 0,\n 'max_depth': 240.0,\n 'max_features': 2,\n 'min_samples_leaf': 0.11044560463320821,\n 'min_samples_split': 0.16792381603680173,\n 'n_estimators': 5}\n"

In [21]:
crit = {0:'entropy',1:'gini'}
feat = {0:'auto',1:'sqrt',2:'log2',3:None}
est = {0:10,1:50,2:300,3:750,4:1200,5:1400}

In [23]:
classifier = RandomForestClassifier(criterion=crit[best['criterion']],max_depth=best['max_depth'],
                                    max_features=feat[best['max_features']],min_samples_leaf=best['min_samples_leaf'],
                                    min_impurity_split=best['min_samples_split'],n_estimators=est[best['n_estimators']])


In [25]:
classifier.fit(X_train,y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/tree/_classes.py:301: FutureWarning: The min_impurity_split parameter is deprecated. Its default value will change from 1e-7 to 0 in version 0.23, and it will be removed in 0.25. Use the min_impurity_decrease parameter instead.
  FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/tree/_classes.py:301: FutureWarning: The min_impurity_split parameter is deprecated. Its default value will change from 1e-7 to 0 in version 0.23, and it will be removed in 0.25. Use the min_impurity_decrease parameter instead.
  FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/tree/_classes.py:301: FutureWarning: The min_impurity_split parameter is deprecated. Its default value will change from 1e-7 to 0 in version 0.23, and it will be removed in 0.25. Use the min_impurity_decrease parameter instead.
  FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/tree/_classes.py:301: FutureWarning: The min_impurity_split parameter is depr

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='entropy', max_depth=240.0,
                       max_features='log2', max_leaf_nodes=None,
                       max_samples=None, min_impurity_decrease=0.0,
                       min_impurity_split=0.16792381603680173,
                       min_samples_leaf=0.11044560463320821,
                       min_samples_split=2, min_weight_fraction_leaf=0.0,
                       n_estimators=1400, n_jobs=None, oob_score=False,
                       random_state=None, verbose=0, warm_start=False)

In [28]:
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report

In [29]:
predictions = classifier.predict(X_test)
print(accuracy_score(predictions,y_test))
print(confusion_matrix(predictions,y_test))
print(classification_report(predictions,y_test))

0.7662337662337663
[[96 25]
 [11 22]]
              precision    recall  f1-score   support

           0       0.90      0.79      0.84       121
           1       0.47      0.67      0.55        33

    accuracy                           0.77       154
   macro avg       0.68      0.73      0.70       154
weighted avg       0.81      0.77      0.78       154

